In [15]:
from youtube.index4 import optimizer
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
import torch, torchvision
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor, Normalize, Resize, RandomHorizontalFlip

In [17]:
transform_train = transforms.Compose([
    Resize(size=(224,224)),
    RandomHorizontalFlip(),
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
transform_test = transforms.Compose([
    Resize(size=(224,224)),
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [23]:
from mydataset import MyDataset
cifa10_dataset = MyDataset(
    csv_file_dir="./cifar-10/trainLabels.csv",
    images_dir="./cifar-10/train",
    transform=transform_train
)

{'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4, 'dog': 5, 'frog': 6, 'horse': 7, 'ship': 8, 'truck': 9}


In [24]:
cifa10_dataset[0]

(tensor([[[-1.1075, -1.1075, -1.1075,  ...,  0.4166,  0.4166,  0.4166],
          [-1.1075, -1.1075, -1.1075,  ...,  0.4166,  0.4166,  0.4166],
          [-1.1075, -1.1075, -1.1075,  ...,  0.4166,  0.4166,  0.4166],
          ...,
          [ 0.9132,  0.9132,  0.9132,  ..., -0.0116, -0.0116, -0.0116],
          [ 0.9132,  0.9132,  0.9132,  ..., -0.0116, -0.0116, -0.0116],
          [ 0.9132,  0.9132,  0.9132,  ..., -0.0116, -0.0116, -0.0116]],
 
         [[-0.9503, -0.9503, -0.9503,  ...,  0.1352,  0.1352,  0.1352],
          [-0.9503, -0.9503, -0.9503,  ...,  0.1352,  0.1352,  0.1352],
          [-0.9503, -0.9503, -0.9503,  ...,  0.1352,  0.1352,  0.1352],
          ...,
          [ 0.4853,  0.4853,  0.4853,  ..., -0.4251, -0.4251, -0.4251],
          [ 0.4853,  0.4853,  0.4853,  ..., -0.4251, -0.4251, -0.4251],
          [ 0.4853,  0.4853,  0.4853,  ..., -0.4251, -0.4251, -0.4251]],
 
         [[-0.7064, -0.7064, -0.7064,  ..., -0.0092, -0.0092, -0.0092],
          [-0.7064, -0.7064,

In [25]:
from torch.utils.data import random_split, DataLoader


In [27]:
N = len(cifa10_dataset)

In [33]:
train_size = int(0.8 * N)
val_size = N - train_size

train_dataset, val_dataset = random_split(
    dataset=cifa10_dataset,
    lengths=[train_size, val_size]
)
len(train_dataset)

40000

In [36]:
BATCH_SIZE = 32
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=2
)
val_loader = torch.utils.data.DataLoader(
    dataset=val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=2
)

In [37]:

import torchvision.models as models

In [38]:
model = models.vgg16(weights = models.VGG16_Weights.IMAGENET1K_V1)
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [47]:
model.classifier[6]

Linear(in_features=4096, out_features=1000, bias=True)

In [44]:
for param in model.features.parameters():
    param.requires_grad = False

In [48]:
import torch.nn as nn
model.classifier[6] = nn.Linear(
    in_features=model.classifier[6].in_features,
    out_features=10
)

In [49]:
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [50]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model.parameters(), lr=0.001, momentum=0.9)